In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np

# 查看数据

In [3]:
train_data = pd.read_csv(
    "./cnews/cnews.train.txt",
    names=['label', 'content'],
    encoding='UTF-8',
    sep='\t')
train_data.head

<bound method NDFrame.head of       label                                            content
0        体育  马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1        体育  商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2        体育  冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3        体育  辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4        体育  揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道  谢亚龙已经被公安...
5        体育  阿的江：八一需重新定位 我们有机会但该进的没进新浪体育讯12月24日，回到主场的北京金隅迎战...
6        体育  姚明未来次节出战成疑 火箭高层称将改变用姚战略新浪体育讯北京时间11月5日消息，休斯敦当地6...
7        体育  姚明：我来承担一切 四连败巨人宣言酷似当年麦蒂新浪体育讯北京时间11月5日消息，据《休斯敦纪...
8        体育  姚麦均无胜殊途同归 活塞酝酿交易火箭不如插一脚新浪体育讯火箭和活塞，是全联盟仅存的两支没有胜...
9        体育  布雷克替补席成功接棒法玛 湖人板凳后卫越打越好新浪体育讯记者戴高乐报道  上赛季，湖人经常在...
10       体育  希伯杜防守体系初见成效 芝加哥铁牛阵已显露真容新浪体育讯记者邱星报道  汤姆-希伯杜，这个名...
11       体育  希伯特由无名小卒变身内线轴心 黑版姚明脱胎换骨新浪体育讯记者高嵩报道  姚明还在寻找比赛的感...
12       体育  投篮打铁防守上当！ 雷霆老三续约遭白眼纯属自找新浪体育讯北京时间11月5日，开拓者主场迎来了...
13       体育  普里兹比拉恢复训练 内线板凳匪徒最迟本月底复出 新浪体育讯记者周超报道  最多23天，普里兹...
14       体育  最强青年军对决雷霆先拔头

## 查看下训练集总共有多少种类别，以及各类别的数量

In [4]:
s = train_data['label'].value_counts()
s

时政    5000
财经    5000
科技    5000
娱乐    5000
房产    5000
游戏    5000
家居    5000
体育    5000
教育    5000
时尚    5000
Name: label, dtype: int64

共10个类别

In [5]:
label2id_dict = {
    x: i
    for i, x in enumerate(train_data['label'].value_counts().index)
}
label2id_dict

{'时政': 0,
 '财经': 1,
 '科技': 2,
 '娱乐': 3,
 '房产': 4,
 '游戏': 5,
 '家居': 6,
 '体育': 7,
 '教育': 8,
 '时尚': 9}

# 数据预处理

## 通过训练集数据，建立我们自己的字典集

In [6]:
from collections import Counter


def build_voca_txt(train_dir, save_path, vocab_size=5000):
    # 创建字典文件
    train_data = pd.read_csv(
        train_dir, names=['label', 'content'], encoding='UTF-8', sep='\t')
    # 对每行的文本内容进行分割
    df = train_data['content'].apply(lambda x: [word for word in x], 1)
    print(df.head())

    all_data = []
    for line in df:
        all_data.extend(line)

    # 对所有字进行统计
    counter = Counter(all_data)
    # 去出出现次数最多前5000-1个
    count_pairs = counter.most_common(vocab_size - 1)
    # 只取字符，不取个数
    words, _ = list(zip(*count_pairs))
    # 添加<PAD>标识符
    words = ['<PAD>'] + list(words)
    #保存到文件里
    open(save_path, mode='w', encoding='utf-8').write('\n'.join(words) + '\n')

In [7]:
build_voca_txt("./cnews/cnews.train.txt", "./cnews/vocab.txt")

0    [马, 晓, 旭, 意, 外, 受, 伤, 让, 国, 奥, 警, 惕,  , 无, 奈, ...
1    [商, 瑞, 华, 首, 战, 复, 仇, 心, 切,  , 中, 国, 玫, 瑰, 要, ...
2    [冠, 军, 球, 队, 迎, 新, 欢, 乐, 派, 对,  , 黄, 旭, 获, 大, ...
3    [辽, 足, 签, 约, 危, 机, 引, 注, 册, 难, 关,  , 高, 层, 威, ...
4    [揭, 秘, 谢, 亚, 龙, 被, 带, 走, ：, 总, 局, 电, 话, 骗, 局, ...
Name: content, dtype: object


## 读取字典的方法，供后续使用

### 转换为{词：id}表示

In [8]:
def read_vocab(vocab_path):
    vocab_dict = {}
    with open(vocab_path, encoding='utf-8') as f:
        for i, word in enumerate(f):
            vocab_dict[word[:-1]] = i
    return vocab_dict

In [9]:
vocab_dict = read_vocab('./cnews/vocab.txt')
print(len(vocab_dict))
print(vocab_dict.items())

5000
dict_items([('<PAD>', 0), ('，', 1), ('的', 2), ('。', 3), ('一', 4), ('是', 5), ('在', 6), ('0', 7), ('有', 8), ('不', 9), ('了', 10), ('中', 11), ('1', 12), ('人', 13), ('大', 14), ('、', 15), ('国', 16), (' ', 17), ('2', 18), ('这', 19), ('上', 20), ('为', 21), ('个', 22), ('“', 23), ('”', 24), ('年', 25), ('学', 26), ('时', 27), ('我', 28), ('地', 29), ('和', 30), ('以', 31), ('到', 32), ('出', 33), ('来', 34), ('会', 35), ('行', 36), ('发', 37), ('：', 38), ('对', 39), ('们', 40), ('要', 41), ('生', 42), ('家', 43), ('他', 44), ('能', 45), ('也', 46), ('业', 47), ('金', 48), ('3', 49), ('成', 50), ('可', 51), ('分', 52), ('多', 53), ('现', 54), ('5', 55), ('就', 56), ('场', 57), ('新', 58), ('后', 59), ('于', 60), ('下', 61), ('日', 62), ('经', 63), ('市', 64), ('前', 65), ('过', 66), ('方', 67), ('得', 68), ('作', 69), ('月', 70), ('最', 71), ('开', 72), ('房', 73), ('》', 74), ('《', 75), ('高', 76), ('9', 77), ('8', 78), ('.', 79), ('而', 80), ('比', 81), ('公', 82), ('4', 83), ('说', 84), (')', 85), ('将', 86), ('(', 87), ('都', 88), ('资', 89),

## 将id文本转换为字符文本

In [10]:
def idtext_decode(idtext, vocab_dict):
    return ' '.join([vocab_dict.get(i, '?') for i in idtext])

## 将字符文本转换为id文本

In [11]:
def process_text(filename, vocab_path, max_length=1000):
    file_data = pd.read_csv(
        filename,
        sep='\t',
        engine='python',
        names=['label', 'content'],
        encoding='UTF-8')

    vocab_id = read_vocab(vocab_path)

    y = file_data['label'].map(label2id_dict)
    X = []

    for words in file_data['content']:
        #         将字符文本转换为id文本
        id_list = [vocab_id[x] for x in words if x in vocab_id]

        #         将文本pad为固定长度
        id_list_length = len(id_list)
        if id_list_length <= max_length:
            id_list.extend([0] * (max_length - id_list_length))  # PAD
        else:
            id_list = id_list[:max_length]
        X.append(id_list)

    return np.array(X), np.array(y)

In [12]:
process_text('./cnews/cnews.train.txt', './cnews/vocab.txt')

(array([[ 387, 1197, 2173, ...,    0,    0,    0],
        [ 199,  964,  280, ...,   14,    2,  101],
        [1108,  581,  157, ...,  709,    2,  381],
        ...,
        [1687,   92,  397, ...,  163,   89,  135],
        [1687,   92,  397, ...,    0,    0,    0],
        [ 779,   25,  193, ...,   72,  358,  253]]),
 array([7, 7, 7, ..., 1, 1, 1], dtype=int64))